In [11]:
import nltk
from nltk.corpus import indian
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import time
import codecs

In [12]:
wordtag_dataset= []
try:
	# read the training data file #
    input_file = codecs.open("dataset.txt", mode = 'r', encoding="utf-8")
    lines = input_file.readlines()
    # pushing words of a line into a list #
    for line in lines:
        line = line.rstrip()
        data = line.split(" ")
        # word,tag=data.split("/")
        wordtag_list=[]
        for wordtag in data:
            try:
                word,tag=wordtag.split("/")
                wordtag_list.append((word,tag))
            except:
                pass
        wordtag_dataset.append(wordtag_list)
        
    input_file.close()
except IOError:
    fo = codecs.open(output_file,mode = 'w',encoding="utf-8")
    fo.write("File not found: {}".format(fin))
    fo.close()
    sys.exit()
print(wordtag_dataset[:3])
        

[[('यह', 'DEM'), ('एशिया', 'NNP'), ('की', 'PSP'), ('सबसे', 'INTF'), ('बड़ी', 'JJ'), ('मस्जिदों', 'NN'), ('में', 'PSP'), ('से', 'PSP'), ('एक', 'QC'), ('है', 'VM'), ('।', 'SYM')], [('इसे', 'PRP'), ('नवाब', 'NN'), ('शाहजेहन', 'NNP'), ('ने', 'PSP'), ('बनवाया', 'VM'), ('था', 'VAUX'), ('।', 'SYM')], [('इसका', 'PRP'), ('प्रवेश', 'NN'), ('द्वार', 'NN'), ('दो', 'QC'), ('मंजिला', 'JJ'), ('है', 'VM'), ('।', 'SYM')]]


In [13]:
hindi_tagged_data = list(indian.tagged_sents('hindi.pos'))
print(hindi_tagged_data[:2])
tagged_data =[ [ tuple for tuple in sent if tuple[1]!=''] for sent in hindi_tagged_data] 




[[('पूर्ण', 'JJ'), ('प्रतिबंध', 'NN'), ('हटाओ', 'VFM'), (':', 'SYM'), ('इराक', 'NNP')], [('संयुक्त', 'NNC'), ('राष्ट्र', 'NN'), ('।', 'SYM')]]


In [14]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(wordtag_dataset,train_size=0.86,test_size=0.14,random_state = 101)

In [15]:
print(len(train_set))
print(len(test_set))

12869
2095


In [16]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

272589
43685


In [17]:
train_tagged_words[:5]

[('सम्मेलन', 'NN'),
 ('की', 'PSP'),
 ('अध्यक्षता', 'NN'),
 ('कर', 'VM'),
 ('रहे', 'VAUX')]

In [18]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}


25
{'NN', 'RDP', 'RB', 'SYM', 'NEG', 'QO', 'QFC', 'INJ', 'CC', 'PSP', 'INTF', 'VAUX', 'NNP', 'RP', 'QC', 'UNK', 'PRP', 'WQ', 'PRPC', 'RBC', 'JJ', 'DEM', 'VM', 'QF', 'NST'}


In [19]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)                           #total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    
    #now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

In [20]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[1.16908260e-01 0.00000000e+00 3.59081291e-03 1.83843002e-02
  1.59352664e-02 3.47498019e-04 4.96425746e-05 0.00000000e+00
  2.33320091e-02 4.87738281e-01 7.94281194e-04 0.00000000e+00
  5.43255247e-02 1.48100341e-02 7.08234031e-03 8.10828700e-04
  1.12026744e-02 1.00939895e-03 9.92851492e-05 6.61900995e-05
  3.79765704e-02 3.62390792e-03 1.94019720e-01 3.72319296e-03
  4.16997634e-03]
 [4.30939227e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.16022103e-01 0.00000000e+00 2.76243091e-02
  5.52486181e-02 2.20994484e-02 3.86740342e-02 0.00000000e+00
  2.76243091e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.39226523e-02 1.65745858e-02 1.60220996e-01 5.52486209e-03
  5.52486209e-03]
 [2.60944217e-01 6.00858359e-03 1.71673822e-03 5.83691001e-02
  1.11587979e-02 5.15021477e-03 0.00000000e+00 0.00000000e+00
  8.58369109e-04 6.86695287e-03 0.00000000e+00 0.00000000e+00
  1.08154505e-01 1.32188842e-01 3.

In [21]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,NN,RDP,RB,SYM,NEG,QO,QFC,INJ,CC,PSP,...,UNK,PRP,WQ,PRPC,RBC,JJ,DEM,VM,QF,NST
NN,0.116908,0.000000,0.003591,0.018384,0.015935,0.000347,0.000050,0.000000,0.023332,0.487738,...,0.000811,0.011203,0.001009,0.000099,0.000066,0.037977,0.003624,0.194020,0.003723,0.004170
RDP,0.430939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.116022,...,0.000000,0.027624,0.000000,0.000000,0.000000,0.093923,0.016575,0.160221,0.005525,0.005525
RB,0.260944,0.006009,0.001717,0.058369,0.011159,0.005150,0.000000,0.000000,0.000858,0.006867,...,0.000858,0.068670,0.001717,0.000000,0.000000,0.109871,0.033476,0.146781,0.012017,0.004292
SYM,0.208749,0.009323,0.013626,0.006013,0.000000,0.003751,0.000055,0.000110,0.065096,0.023777,...,0.000110,0.182601,0.001434,0.000221,0.000000,0.063827,0.056766,0.007668,0.007613,0.006013
NEG,0.002885,0.000000,0.000577,0.007501,0.000577,0.000000,0.000000,0.000000,0.002885,0.000000,...,0.000000,0.000577,0.000000,0.000000,0.000000,0.006924,0.000577,0.956722,0.000000,0.000000
QO,0.767281,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027650,0.002304,...,0.000000,0.004608,0.000000,0.000000,0.000000,0.050691,0.009217,0.002304,0.002304,0.066820
QFC,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.642857,0.000000
INJ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000
CC,0.270273,0.000000,0.008443,0.017279,0.000491,0.003829,0.000098,0.000000,0.012174,0.000098,...,0.000000,0.187414,0.005301,0.000393,0.000000,0.106028,0.046436,0.006185,0.014824,0.005989
PSP,0.372791,0.000000,0.007348,0.003509,0.002632,0.003327,0.000110,0.000018,0.001188,0.082107,...,0.000530,0.046226,0.001225,0.000311,0.000165,0.111115,0.020509,0.112852,0.016707,0.048749


In [22]:
T = list(set([pair[1] for pair in train_tagged_words]))

def Viterbi(words, train_bag = train_tagged_words):
    state = []
    global T    
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['NNP', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [23]:
txt = 'इराक के विदेश मंत्री ने अमरीका के उस प्रस्ताव का मजाक उड़ाया है , जिसमें अमरीका ने संयुक्त राष्ट्र के प्रतिबंधों को इराकी नागरिकों के लिए  कम हानिकारक बनाने के लिए कहा है ।'
txt = txt.split(" ")
print(Viterbi(txt))

[('इराक', 'NNP'), ('के', 'PSP'), ('विदेश', 'NN'), ('मंत्री', 'NNP'), ('ने', 'PSP'), ('अमरीका', 'NNP'), ('के', 'PSP'), ('उस', 'DEM'), ('प्रस्ताव', 'NN'), ('का', 'PSP'), ('मजाक', 'NN'), ('उड़ाया', 'VM'), ('है', 'VAUX'), (',', 'SYM'), ('जिसमें', 'PRP'), ('अमरीका', 'NNP'), ('ने', 'PSP'), ('संयुक्त', 'JJ'), ('राष्ट्र', 'NN'), ('के', 'PSP'), ('प्रतिबंधों', 'NN'), ('को', 'PSP'), ('इराकी', 'JJ'), ('नागरिकों', 'NN'), ('के', 'PSP'), ('लिए', 'PSP'), ('', 'SYM'), ('कम', 'QF'), ('हानिकारक', 'JJ'), ('बनाने', 'VM'), ('के', 'PSP'), ('लिए', 'PSP'), ('कहा', 'VM'), ('है', 'VAUX'), ('।', 'SYM')]


In [26]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [27]:
print(test_tagged_words)

['हुसैन', 'को', 'हवाई', 'अड्डे', 'के', 'आव्रजन', 'क्षेत्र', 'में', 'रोक', 'रखा', 'है', 'और', 'उसे', 'अभी', 'गिरफ्तार', 'नहीं', 'किया', 'गया', 'है', '।', 'मुशर्रफ', 'ने', 'कहा', 'कि', 'अगर', 'मुस्लिम', 'देश', 'या', 'भारत', 'इराक', 'में', 'अपनी', 'सेना', 'भेजते', 'हैं', 'तो', 'पाकिस्तान', 'भी', 'पीछे', 'नहीं', 'रहेगा', '।', 'उन्होंने', 'कहा', 'कि', 'अमेरिका', 'पाकिस्तान', 'के', 'उसके', 'पड़ोसी', 'मुल्कों', 'के', 'साथ', 'बेहतर', 'रिश्ते', 'की', 'वकालत', 'करता', 'रहा', 'है', '।', 'सरकार', 'उन्हें', 'जाल', 'और', 'नौकाएं', 'उपलब्ध', 'कराने', 'के', 'लिए', 'तुरंत', 'कदम', 'उठा', 'रही', 'है', '।', 'इसके', 'लिए', '१०१', 'रुपये', 'और', '५१', 'रुपये', 'भुगतान', 'करना', 'होगा', '।', 'शनिवार', 'को', 'नई', 'दिल्ली', 'में', 'पार्टी', 'की', 'एक', 'बैठक', 'में', 'उन्होंने', 'कहा', 'कि', 'केंद्र', 'की', 'सत्तारूढ़', 'यूपीए', 'सरकार', 'का', 'यह', 'प्रस्ताव', 'इस', 'बात', 'को', 'दर्शाता', 'है', 'कि', 'देश', 'में', 'सुरक्षा', 'की', 'स्थिति', 'कितनी', 'दयनीय', 'है', '?', 'राज्यसभा', 'में', 'प्रश्नकाल', 'के', '

In [28]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  585.578507900238
Viterbi Algorithm Accuracy:  94.58128078817734
